In [1]:
import pandas as pd

In [3]:
from functools import partial

In [7]:
import sys

In [8]:
sys.path.append('/Users/pradap/Documents/Research/Python-Package/scaling/dmagellan/')

In [58]:
from dmagellan.core.stringcontainer import StringContainer
from dmagellan.core.utils import str2bytes
from dmagellan.core.whitespacetokenizer import WhiteSpaceTokenizer
from dmagellan.core.attrequivblocker import block_tables_dk

In [27]:
import string
def remove_stopwords(tokens, stopwords):
    otokens = []
    for token in tokens:
        if not stopwords.has_key(token):
            otokens.append(token)
    return otokens
def process_col(column, stopwords):
    column = column.str.translate(None, string.punctuation)
    column = column.str.lower()
    if stopwords != None:
        stopword_dict = dict(zip(stopwords, [0]*len(stopwords)))
        prem_stopwords = partial(remove_stopwords, stopwords=stopword_dict)
        tmp = column.str.split()
        tmp = tmp.map(prem_stopwords)
        tmp = tmp.str.join(' ')
        column = tmp
    return column
    
def preprocess_table(dataframe, overlap_attr, id_attr, stopwords=None):
    objsc = StringContainer()
    projdf = dataframe[[overlap_attr, id_attr]]
    projdf = projdf.dropna()
    projdf[overlap_attr] = process_col(projdf[overlap_attr], stopwords)
    for row in projdf.itertuples():
        val = str2bytes(row[1])
        uid = row[-1]
        objsc.push_back(uid, val)
    return objsc


In [48]:
A = pd.read_csv('../datasets/person_table_A.csv')
B = pd.read_csv('../datasets/person_table_B.csv')

In [50]:
help(block_tables_sm)

Help on function block_tables_sm in module dmagellan.core.attrequivblocker:

block_tables_sm(A, B, l_key, r_key, l_attr, r_attr, l_out=None, r_out=None, l_prefix='l_', r_prefix='r_', nlchunks=1, nrchunks=1)



In [54]:
A.head()

,ID,name,birth_year,hourly_wage,address,zipcode
0,0,Kevin Smith,1989,30.0,"607 From St, San Francisco",94107
1,1,Michael Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
2,2,William Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
3,3,Binto George,1987,32.5,"423 Powell St, San Francisco",94122
4,4,Alphonse Kemper,1984,35.0,"1702 Post Street, San Francisco",94122


In [63]:
C = block_tables_sm(A, B, 'ID', 'ID', 'zipcode', 'zipcode', ['zipcode'], ['zipcode'])

In [64]:
C

,_id,l_ID,r_ID,l_zipcode,r_zipcode
0,0,0,0,94107,94107
1,1,0,1,94107,94107
2,2,0,5,94107,94107
3,3,2,0,94107,94107
4,4,2,1,94107,94107
5,5,2,5,94107,94107
6,6,1,2,94122,94122
7,7,1,3,94122,94122
8,8,1,4,94122,94122
9,9,3,2,94122,94122


In [37]:
%time x = preprocess_table(A, 'title', 'id', stopwords=['is', 'on', 'the'])

CPU times: user 4.19 s, sys: 293 ms, total: 4.48 s
Wall time: 4.54 s


In [41]:
y = pd.DataFrame([[1, 2], [2, 3]], columns=['lid', 'rid'])

In [73]:
ws = WhiteSpaceTokenizer()

In [70]:
x = pd.DataFrame()

In [74]:
x['a'] = A.address.map(str2bytes).map(ws.tokenize)
x['b'] = B.address.map(str2bytes).map(ws.tokenize)

In [80]:
a1 = x.ix[0]['a']
b1 = x.ix[0]['b']

In [82]:
help(x.apply)

Help on method apply in module pandas.core.frame:

apply(self, func, axis=0, broadcast=False, raw=False, reduce=None, args=(), **kwds) method of pandas.core.frame.DataFrame instance
    Applies function along input axis of DataFrame.
    
    Objects passed to functions are Series objects having index
    either the DataFrame's index (axis=0) or the columns (axis=1).
    Return type depends on whether passed function aggregates, or the
    reduce argument if the DataFrame is empty.
    
    Parameters
    ----------
    func : function
        Function to apply to each column/row
    axis : {0 or 'index', 1 or 'columns'}, default 0
        * 0 or 'index': apply function to each column
        * 1 or 'columns': apply function to each row
    broadcast : boolean, default False
        For aggregation functions, return object of same size with values
        propagated
    raw : boolean, default False
        If False, convert each row or column into a Series. If raw=True the
        pass

In [67]:
def get_proj_cols(idcol, attr, out):
    ocols = [idcol, attr]
    if out != None:
        out = [c for c in out if c not in ocols]
    ocols.extend(out)
    return ocols

In [69]:
get_proj_cols('ID', 'zipcode', ['zipcode'])

['ID', 'zipcode']

In [87]:
def fn(x):
    return len(np.intersect1d(x[0], x[1]))

In [105]:
def fn1(x, threshold):
    return len(set(x[0]).intersection(x[1])) >= threshold
    

In [85]:
import numpy as np

In [106]:
from functools import partial

In [110]:
pfn = partial(fn1, threshold=3)

In [111]:
x.apply(pfn, axis=1, raw=True)

0     True
1     True
2     True
3    False
4     True
dtype: bool

In [112]:
X = pd.read_csv('../datasets/sample_songs.csv')
Y = pd.read_csv('../datasets/sample_tracks.csv')

In [117]:
len(X.title)

100000

In [115]:
len(X.title.dropna())

99999

In [ ]:
import py_stringsimjoin as ssj